In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re

In [17]:
#path for the file
file_path="pizza.txt"

#reading the file
with open (file_path,"r") as file:
    text=file.read()

    #spliting the data input according to the regex provided
    data=[sentence.strip() for sentence in re.split(r'(?<=[.!?]\s+)',text) if sentence.strip()]
print(data)


['Pizza, the delectable and iconic dish that has transcended borders and captivated taste buds worldwide, is a testament to the extraordinary fusion of flavors, creativity, and cultural significance.', 'Originating from the sun-kissed lands of Italy, pizza has evolved into an art form that unites people from diverse backgrounds in a shared love for its mouthwatering combinations.', 'Its history stretches back centuries, with roots tracing back to ancient civilizations like the Greeks, Romans, and Egyptians, who all had their versions of flatbreads adorned with various ingredients.', 'However, it was the vibrant city of Naples, Italy, that birthed the pizza we know and adore today.', 'With its soft and chewy Neapolitan crust, topped with the perfect balance of tomatoes, mozzarella cheese, and fresh basil, the Margherita pizza pays homage to Queen Margherita of Italy and embodies the colors of the Italian flag.', 'As pizza migrated from the shores of Naples, it found its way to the Unite

In [19]:
#making an instance of the tokenizer
# tokenizer is used to map each unique word into uniqye integer
tokenizer=Tokenizer()

#scans through all the texts and keeps track of the frequency of each word in the texts
tokenizer.fit_on_texts(data)
# +1 is to act as a place holder for out-of-vocabulary
total_words=len(tokenizer.word_index)+1
print(total_words)

687


In [20]:
input_sequences=[]
#making n-gram list for input text
for line in data:
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)
print(input_sequences)

[[3, 1], [3, 1, 233], [3, 1, 233, 2], [3, 1, 233, 2, 45], [3, 1, 233, 2, 45, 35], [3, 1, 233, 2, 45, 35, 13], [3, 1, 233, 2, 45, 35, 13, 8], [3, 1, 233, 2, 45, 35, 13, 8, 114], [3, 1, 233, 2, 45, 35, 13, 8, 114, 234], [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2], [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235], [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74], [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74, 115], [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74, 115, 236], [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74, 115, 236, 14], [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74, 115, 236, 14, 5], [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74, 115, 236, 14, 5, 116], [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74, 115, 236, 14, 5, 116, 6], [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74, 115, 236, 14, 5, 116, 6, 1], [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74, 115, 236, 14, 5, 116, 6, 1, 237], [3, 1, 233, 2, 45, 35, 13, 8, 114, 234, 2, 235, 74, 115, 2

In [22]:
max_sequence_len=max([len(seq) for seq in input_sequences],default=0)
# padding the sequence to take the every input to the same length
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding="pre"))

# choosing X,y
# X = choosing all the list inside the list except the last item of each list
# y = choosing all the last item of the each list
# by this we can predict the next word(Integer) using the sequence of words (Integers)
X,y=input_sequences[:,:-1],input_sequences[:,-1]

In [23]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)